# Numerical Optimization using PyTorch

In this Jupyter notebook, I perform some basic econometric optimization routines using the PyTorch package.


In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import torch
import torch.optim as optim
from torch.autograd import Variable
import torch.distributions as td
import mdmm

## A Basic Optimization Problem

Consider the following problem:

$$
    \min_{x} \quad 2 x^2 - 7 x + 6
$$

We use the `torch.tensor` method to store parameters, constants and matrices. However, the parameters over which we are optimizing the objective function require an additional option `requires_grad=True` so that PyTorch knows how to perform the backpropagation.

In [2]:
x = Variable(torch.ones(1), requires_grad=True)

optimizer = optim.Adam([x], lr=0.05)

for _ in range(1000):
    optimizer.zero_grad()
    y = (2 * x**2 - 7 * x + 6)
    y.backward()
    optimizer.step()

print("Numerical optimization solution:", x)
print("Analytic optimization solution:", 7/4)

Numerical optimization solution: tensor([1.7500], requires_grad=True)
Analytic optimization solution: 1.75


`optimizer.step()` moves the parameter in the direction that minimizes the objective function using the gradient computed in the `.backward()` operation.

$$
\quad
$$

----------------------------------------------------------

## Maximum Likelihood Estimation Part 1

In this example, consider some observations $\{X_i \mid i = 1, \ldots, N\}$ drawn from a normal distribution with mean $\mu$ and variance $\sigma^2$.

In [3]:
N = 1000
μ, σ = 0.05, 0.1
x_data = μ + torch.randn(N) * σ

We know that the ML estimates maximize the log-likelihood function, or correspondingly minimize the negative log-likelihood. We can set the mean and standard deviation as PyTorch `Variable`s and ask the optimizer to minimize the objective accordingly.

$$
    \widehat{\boldsymbol{\theta}} \; \equiv \; \begin{pmatrix} \widehat{\mu} \\ \widehat{\sigma} \end{pmatrix} \quad \in \;\; \underset{\begin{pmatrix} \mu \\ \sigma \end{pmatrix}}{\arg \min} -\frac{N}{2} \log (2 \pi) - N \log \sigma - \frac{1}{2} \sum_{i = 1}^N  {\left( \frac{X_i - \mu}{\sigma} \right)}^2
$$

I used `torch.distribution`'s inbuilt log-likelihood method corresponding to a Normal distribution to define the sum of the log-likelihoods over the set of observations.

The subpackage `torch.optim` implements various optimization algorithms. To construct an `Optimizer`, we provide an iterable containing the parameters (here, $\mu$ and $\sigma$) to optimize. Then, we can specify optimizer-specific options such as the learning rate, weight decay, etc. I chose the Adam algorithm with a small learning rate.

In [4]:
# Set parameters for the ML optimization.
mu_hat = Variable(torch.zeros(1), requires_grad=True)
sigma_hat = Variable(torch.ones(1), requires_grad=True)

# Define the objective function.
def log_lik(mu, sigma):

    return td.Normal(loc=mu, scale=sigma).log_prob(x_data).sum()

# Define the adaptive gradient descent optimizer used to find the estimates.
opt = optim.Adam([mu_hat, sigma_hat], lr=0.05)

In [5]:
for epoch in range(10000):

    opt.zero_grad() # Reset gradient inside the optimizer

    # Compute the objective at the current parameter values.
    loss = - log_lik(mu_hat, sigma_hat)
    loss.backward() # Gradient computed.
    opt.step()      # Update parameter values using gradient descent.

In [6]:
print('Parameters: mu = {:10.3e}, sigma = {:10.3e}'.format(
    mu_hat.detach().numpy()[0], sigma_hat.detach().numpy()[0])
)

Parameters: mu =  4.852e-02, sigma =  9.745e-02


We know that the asymptotic distribution of the ML estimator is given by

$$
    \sqrt{N} \left(\widehat\boldsymbol{\theta} - \boldsymbol{\theta} \right) \;\; \underset{d}{\longrightarrow} \;\; \mathcal{N}\left( \boldsymbol{0} \, , \, \mathbf{I}(\boldsymbol{\theta})^{-1} \right) \;\; \Longrightarrow \;\; \widehat\boldsymbol{\theta} \; \underset{d}{\sim} \;\; \mathcal{N}\left( \boldsymbol{\theta} \, , \, \frac{\mathbf{I}(\boldsymbol{\theta})^{-1}}{N}  \right)
$$

Since the Fisher information matrix is the inverse of the Hessian of the log-likelihood, we can use `torch.autograd.functional.hessian` to derive the Hessian matrix, which yields the information matrix and then the standard errors by taking the square root of the diagonal elements.

In [7]:
theta_hat = (mu_hat, sigma_hat)

# Fisher Information matrix.
info_mle = -torch.tensor(torch.autograd.functional.hessian(log_lik, theta_hat))

# Compute variance matrix.
var_mle = torch.inverse(info_mle)/N

# Compute standard errors.
std_mle = np.sqrt(np.diag(var_mle))

In [8]:
print('Standard Errors: mu = {:10.3e}, sigma = {:10.3e}'.format(std_mle[0], std_mle[1]))

Standard Errors: mu =  9.745e-05, sigma =  6.955e-05


----------------------------------------------------------

## Ordinary Least Squares Estimation

In this next example, consider some observations $\{Y_i, \mathbf{X}_i \mid i = 1, \ldots, N\}$ corresponding to the following DGP:

$$
    Y_i \; = \; \mathbf{X}_i \boldsymbol{\beta} + \varepsilon_i
$$

where we assume that $\varepsilon_i \overset{\text{i.i.d.}}{\sim} \mathcal{N}(0, \sigma^2)$.

I simulate a dataset with 3 covariates below, and will attempt to recover the coefficients of the linear using the OLS objective function.

In [9]:
## Simulation of data corresponding to the DGP above

mean_X = torch.tensor([5.0, -3.0, 2.0])
cov_X = torch.tensor(
    [[4.0, 1.6, -2.4],
     [1.6, 1.0, -0.9],
     [-2.4, -0.9, 9.0]]
)
N = 1000
sigma_eps = 0.9

X = td.MultivariateNormal(
    loc=mean_X, covariance_matrix=cov_X).sample((N,))
eps = td.Normal(loc=0, scale=sigma_eps).sample((N,))

# Add a column of ones
X = torch.column_stack((torch.ones(N,), X))

# Choose linear model coefficients
beta = torch.tensor([10, -1.0, 0.5, 2.0])

y = X @ beta + eps

In [10]:
# Parameter and objective function for OLS

beta_hat = Variable(torch.zeros(4), requires_grad=True)

def ols_loss(b):
    return torch.nn.MSELoss()(X @ b, y)

opt_ols = optim.Adam([beta_hat], lr=0.05)

In [11]:
# Optimizing over the OLS loss function

for epoch in range(5000):

    opt_ols.zero_grad() # Reset gradient inside the optimizer

    # Compute the objective at the current parameter values.
    loss = ols_loss(beta_hat)
    loss.backward() # Gradient computed.
    opt_ols.step()      # Update parameter values using gradient descent.

In [12]:
# Printing the minimizer values

np.array(beta_hat.detach())

array([10.318044 , -1.0214003,  0.5652391,  1.9927149], dtype=float32)

As a sanity check, we can check the results obtained using the standard regression package `statsmodels`.

In [13]:
sm.OLS(np.array(y), np.array(X)).fit().params

array([10.318071 , -1.021403 ,  0.5652436,  1.9927149], dtype=float32)

Similary, we can compute and compare the standard errors.

In [14]:
ols_r = (X @ beta_hat - y).detach()
sig2_hat = (ols_r @ ols_r.T)/(X.shape[0] - X.shape[1])

In [15]:
np.sqrt(np.diag(sig2_hat * torch.inverse(X.T @ X)))

array([0.2442146 , 0.02346404, 0.04540195, 0.00982629], dtype=float32)

In [16]:
sm.OLS(np.array(y), np.array(X)).fit().bse

array([0.24421541, 0.02346409, 0.04540214, 0.00982628], dtype=float32)

----------------------------------------------------------

## Constrained Optimization

$$
    \min_{x, y} \;\; 5x - 3 y \quad \text{s.t.} \;\; x^2 + y^2 = 136
$$

In [40]:
# Set parameters for the constrained optimization.
x = Variable(torch.zeros(1), requires_grad=True)
y = Variable(torch.ones(1), requires_grad=True)
λ = Variable(torch.ones(1), requires_grad=True)

opt_con = optim.Adam([x, y], lr=0.05)

for _ in range(10000):
    opt_con.zero_grad()
    objective = 5*x - 3*y - 0.01 * (x ** 2 + y ** 2 - 136)
    objective.backward()
    opt_con.step()

print("Numerical optimization solution: x = {} and y = {}".format(x, y))
print("Analytic optimization solution: x = -10 and y = 6")

Numerical optimization solution: x = tensor([-545.6191], requires_grad=True) and y = tensor([560.3257], requires_grad=True)
Analytic optimization solution: x = -10 and y = 6


tensor([-499.4106], requires_grad=True)